In [1]:
import pandas as pd
from glob import glob
import os
import sys
cnn_scripts_path = '/home/csutter/DRIVE-clean/CNN/scripts'
sys.path.append(cnn_scripts_path)

# imported reusable code from CNN directory
import results_summaries

### CNN

In [45]:
files_list = glob("/home/csutter/DRIVE-clean/CNN/data_preds/*")
files_list = [x for x in files_list if "A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue" in x]

# OG WAY: A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue
# NEW WAY: A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue
print(len(files_list))

30


In [52]:
# df = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"] # change this according to phase
    dfs30.append(d)

df = pd.concat(dfs30).reset_index()

print(len(df))
display(df.head(4))



107445


,index,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,innerPhase,Unnamed: 0,outerPhase,img_name,img_orig,...,timeofevent,innerPhaseOrig,prob_dry,prob_poor_viz,prob_snow,prob_snow_severe,prob_wet,model_pred,tracker,model_prob
0,0,0,0,0,0,innerVal,0,inner5CV,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,...,"dawn:12:06:29.361832, sunrise:12:37:47.866183,...",innerVal,0.033092,0.000242,0.003411,0.000642,0.962614,wet,OT2_m1_T3V4,0.962614
1,1,1,1,1,1,innerVal,1,inner5CV,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,...,"dawn:12:06:29.361832, sunrise:12:37:47.866183,...",innerVal,0.049065,0.000215,0.009169,0.000536,0.941015,wet,OT2_m1_T3V4,0.941015
2,2,2,2,2,2,innerVal,2,inner5CV,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,...,"dawn:12:06:29.361832, sunrise:12:37:47.866183,...",innerVal,0.056126,0.000282,0.014468,0.000902,0.928222,wet,OT2_m1_T3V4,0.928222
3,3,3,3,3,3,innerVal,3,inner5CV,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,...,"dawn:12:06:29.361832, sunrise:12:37:47.866183,...",innerVal,0.047258,0.000116,0.014778,0.000395,0.937453,wet,OT2_m1_T3V4,0.937453


In [53]:
df.columns

Index(['index', 'Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1',
       'innerPhase', 'Unnamed: 0', 'outerPhase', 'img_name', 'img_orig',
       'site', 'img_index', 'img_cat', 'foldnum', 'ID', 'Latitude',
       'Longitude', 'date', 'yr', 'mo', 'day', 'time', 'yr_mo', 'timeofday',
       'timeofevent', 'innerPhaseOrig', 'prob_dry', 'prob_poor_viz',
       'prob_snow', 'prob_snow_severe', 'prob_wet', 'model_pred', 'tracker',
       'model_prob'],
      dtype='object')

In [54]:
dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")

print(dict_i)

df = pd.DataFrame(dict_i,index=[0])


df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

display(df)


{'totalims': 107445, 'correctims': 74267, 'nims_snow_severe': 7635, 'correct_snow_severe': 5909, 'nims_snow': 14945, 'correct_snow': 9014, 'nims_wet': 42290, 'correct_wet': 30412, 'nims_dry': 38000, 'correct_dry': 25337, 'nims_poor_viz': 4575, 'correct_poor_viz': 3595, 'ok': 97482, 'exp_desc': '', 'exp_details': '', 'tracker': ''}


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,107445,74267,7635,5909,14945,9014,42290,30412,38000,25337,...,,,,0.691209,0.666763,0.785792,0.603145,0.773936,0.71913,0.709753


In [ ]:
# outerTest 68.1
# innerVal 69.1

### Downstream

In [55]:
files_list = glob("/home/csutter/DRIVE-clean/downstream/data_predsFinal/*")

files_list = [x for x in files_list if "A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue" in x]

# A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse
# A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue
# A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue

# add this additional subbsetting if needed
# subset_downstream = "{'max_depth': 10, 'max_samples': 1.0, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}"
# files_list = [x for x in files_list if subset_downstream in x]

# OG WAY: A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue
# NEW WAY: A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue
print(len(files_list))

print(len(files_list))

30
30


In [58]:
# df = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"]# change this according to phase
    d["model_pred"] = d["ds_pred"]
    dfs30.append(d)

df = pd.concat(dfs30).reset_index()

print(len(df))
display(df.head(4))


107445


,index,Unnamed: 0.1,innerPhase,outerPhase,img_name,img_orig,site,img_cat,foldnum,timeofday,...,isobaricInhPa,uavg,ds_prob_dry,ds_prob_poor_viz,ds_prob_snow,ds_prob_snow_severe,ds_prob_wet,ds_pred,ds_prob,model_pred
0,459,459,innerVal,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,snow_severe,0,night,...,500.0,5.797924,0.002869,0.031958,0.104073,0.859510,0.001590,snow_severe,0.859510,snow_severe
1,460,460,innerVal,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,snow_severe,0,night,...,500.0,5.468119,0.001824,0.013866,0.076977,0.906804,0.000529,snow_severe,0.906804,snow_severe
2,461,461,innerVal,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,snow_severe,0,day,...,500.0,4.567119,0.035991,0.001589,0.620580,0.179289,0.162552,snow,0.620580,snow
3,462,462,innerVal,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,snow_severe,0,day,...,500.0,4.894132,0.067214,0.000568,0.355714,0.169399,0.407106,wet,0.407106,wet


In [59]:
dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")

print(dict_i)

df = pd.DataFrame(dict_i,index=[0])


df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

display(df)


{'totalims': 107445, 'correctims': 86667, 'nims_snow_severe': 7635, 'correct_snow_severe': 6107, 'nims_snow': 14945, 'correct_snow': 11147, 'nims_wet': 42290, 'correct_wet': 32213, 'nims_dry': 38000, 'correct_dry': 33859, 'nims_poor_viz': 4575, 'correct_poor_viz': 3341, 'ok': 102105, 'exp_desc': '', 'exp_details': '', 'tracker': ''}


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,107445,86667,7635,6107,14945,11147,42290,32213,38000,33859,...,,,,0.806617,0.891026,0.730273,0.745868,0.799869,0.761717,0.785751


In [ ]:
# outerTest 80.2
# innerVal 80.7

### Downstream - High vs low quality
For reference: /home/csutter/DRIVE/weather_img_concatmodels/cnn_hrrr_fcsthr2/nested_ensemble_adhocsummaries.ipynb

In [39]:
Low_quality_kept_half = ['Skyline_1790',
'Skyline_6390',
'Skyline_6160',
'Skyline_5630',
'Skyline_6117',
'Skyline_6163',
'Skyline_6493']

Low_quality_all = ['Skyline_1803',
'Skyline_1790',
'Skyline_6390',
'Skyline_6160',
'Skyline_6048',
'Skyline_5630',
'Skyline_6117',
'Skyline_6163',
'Skyline_6419',
'Skyline_2103',
'Skyline_6294',
'Skyline_8344',
'Skyline_6493',
'Skyline_1824',
'Skyline_1873']

In [41]:
files_list = glob("/home/csutter/DRIVE-clean/downstream/data_predsFinal/*")

files_list = [x for x in files_list if "A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue" in x]


#### High Quality
dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"]# change this according to phase
    d = d[~d["site"].isin(Low_quality_all)] # Note the ~
    d["model_pred"] = d["ds_pred"]
    dfs30.append(d)
df = pd.concat(dfs30).reset_index()

print(len(df))
dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")
df = pd.DataFrame(dict_i,index=[0])
df["overall"] = df["correctims"]/df["totalims"]
for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]
df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

print("HIGH QUALITY")
display(df)

#### Low Quality
dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"]# change this according to phase
    d = d[d["site"].isin(Low_quality_all)]
    d["model_pred"] = d["ds_pred"]
    dfs30.append(d)
df = pd.concat(dfs30).reset_index()

print(len(df))
dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")
df = pd.DataFrame(dict_i,index=[0])
df["overall"] = df["correctims"]/df["totalims"]
for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]
df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

print("LOW QUALITY")
display(df)

61685
HIGH QUALITY


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,61685,52765,5595,4703,8530,6652,23440,19583,20160,18811,...,,,,0.855394,0.933085,0.761616,0.779836,0.840572,0.835452,0.830112


45760
LOW QUALITY


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,45760,33902,2040,1404,6415,4495,18850,12630,17840,15048,...,,,,0.740865,0.843498,0.528455,0.700701,0.688235,0.670027,0.686183


### Experiments
- Side experiments for manuscript regarding generalizability and other takeaways

#### Experiment 1: OneTrain
Proving validity of downstream flow (training on a diff dataset). This side experiment is then the alternative, which is training on the same dataset for both CNN and RF, calling this oneTrain experiment.

CNN

In [20]:
files_list = glob("/home/csutter/DRIVE-clean/side_experiments_data_and_models/CNN/data_preds_expOneTrain/*")
files_list = [x for x in files_list if "A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue" in x]
# "A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse"
# "A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue"
# "A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue"
print(len(files_list))

30


In [21]:
files_list[0]

'/home/csutter/DRIVE-clean/side_experiments_data_and_models/CNN/data_preds_expOneTrain/nestcv_5cat_onetrain_OT5_m4_T4V0__A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue.csv'

In [22]:
de = pd.read_csv('/home/csutter/DRIVE-clean/side_experiments_data_and_models/CNN/data_preds_expOneTrain/nestcv_5cat_onetrain_OT5_m0_T0V1__A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse.csv')

de.groupby(["innerPhase"]).count()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,outerPhase,img_name,img_orig,site,img_index,img_cat,...,timeofevent,innerPhaseOrig,prob_dry,prob_poor_viz,prob_snow,prob_snow_severe,prob_wet,model_pred,tracker,model_prob
innerPhase,,,,,,,,,,,,,,,,,,,,,
NAOuterTest,3576,3576,3576,3576,3576,3576,3576,3576,3576,3576,...,3576,3576,3576,3576,3576,3576,3576,3576,3576,3576
innerTrain,14352,14352,14352,14352,14352,14352,14352,14352,14352,14352,...,14352,14352,14352,14352,14352,14352,14352,14352,14352,14352
innerVal,3561,3561,3561,3561,3561,3561,3561,3561,3561,3561,...,3561,3561,3561,3561,3561,3561,3561,3561,3561,3561


In [23]:
# df = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"] # change this according to phase
    dfs30.append(d)

df = pd.concat(dfs30).reset_index()

print(len(df))
display(df.head(4))



107445


,index,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,innerPhase,Unnamed: 0,outerPhase,img_name,img_orig,site,...,timeofevent,innerPhaseOrig,prob_dry,prob_poor_viz,prob_snow,prob_snow_severe,prob_wet,model_pred,tracker,model_prob
0,459,459,459,459,innerVal,459,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,...,"dawn:11:29:00.847281, sunrise:11:59:10.264774,...",innerVal,0.002666,0.050255,0.002159,0.940243,0.004678,snow_severe,nestcv_5cat_onetrain_OT5_m4_T4V0,0.940243
1,460,460,460,460,innerVal,460,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,...,"dawn:11:29:00.847281, sunrise:11:59:10.264774,...",innerVal,0.001996,0.020671,0.002161,0.972410,0.002762,snow_severe,nestcv_5cat_onetrain_OT5_m4_T4V0,0.972410
2,461,461,461,461,innerVal,461,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,...,"dawn:11:29:00.847281, sunrise:11:59:10.264774,...",innerVal,0.037506,0.000784,0.818759,0.103861,0.039091,snow,nestcv_5cat_onetrain_OT5_m4_T4V0,0.818759
3,462,462,462,462,innerVal,462,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,...,"dawn:11:29:00.847281, sunrise:11:59:10.264774,...",innerVal,0.130350,0.002081,0.510149,0.254475,0.102945,snow,nestcv_5cat_onetrain_OT5_m4_T4V0,0.510149


In [24]:
dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")

print(dict_i)

df = pd.DataFrame(dict_i,index=[0])


df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

display(df)


{'totalims': 107445, 'correctims': 77104, 'nims_snow_severe': 7635, 'correct_snow_severe': 6239, 'nims_snow': 14945, 'correct_snow': 9608, 'nims_wet': 42290, 'correct_wet': 30787, 'nims_dry': 38000, 'correct_dry': 26821, 'nims_poor_viz': 4575, 'correct_poor_viz': 3649, 'ok': 98746, 'exp_desc': '', 'exp_details': '', 'tracker': ''}


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,107445,77104,7635,6239,14945,9608,42290,30787,38000,26821,...,,,,0.717614,0.705816,0.797596,0.642891,0.817158,0.727997,0.738291


Downstream

In [25]:
files_list = glob("/home/csutter/DRIVE-clean/side_experiments_data_and_models/downstream/data_preds_expOneTrain/*")
files_list = [x for x in files_list if "A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue" in x]
# A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue
# A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse

print(len(files_list))


30


In [26]:
# df = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"]# change this according to phase
    d["model_pred"] = d["ds_pred"]
    dfs30.append(d)

df = pd.concat(dfs30).reset_index()

print(len(df))
display(df.head(4))


107445


,index,Unnamed: 0.1,innerPhase,outerPhase,img_name,img_orig,site,img_cat,foldnum,timeofday,...,isobaricInhPa,uavg,ds_prob_dry,ds_prob_poor_viz,ds_prob_snow,ds_prob_snow_severe,ds_prob_wet,ds_pred,ds_prob,model_pred
0,459,459,innerVal,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,snow_severe,0,night,...,500.0,5.797924,0.000131,0.000979,0.003921,0.994951,0.000017,snow_severe,0.994951,snow_severe
1,460,460,innerVal,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,snow_severe,0,night,...,500.0,5.468119,0.000239,0.000000,0.002644,0.997104,0.000014,snow_severe,0.997104,snow_severe
2,461,461,innerVal,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,snow_severe,0,day,...,500.0,4.567119,0.002370,0.001546,0.982409,0.001933,0.011741,snow,0.982409,snow
3,462,462,innerVal,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,snow_severe,0,day,...,500.0,4.894132,0.002789,0.000026,0.982128,0.002827,0.012230,snow,0.982128,snow


In [27]:
dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")

print(dict_i)

df = pd.DataFrame(dict_i,index=[0])


df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

display(df)


{'totalims': 107445, 'correctims': 84301, 'nims_snow_severe': 7635, 'correct_snow_severe': 6059, 'nims_snow': 14945, 'correct_snow': 10353, 'nims_wet': 42290, 'correct_wet': 33933, 'nims_dry': 38000, 'correct_dry': 30547, 'nims_poor_viz': 4575, 'correct_poor_viz': 3409, 'ok': 100665, 'exp_desc': '', 'exp_details': '', 'tracker': ''}


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,107445,84301,7635,6059,14945,10353,42290,33933,38000,30547,...,,,,0.784597,0.803868,0.745137,0.69274,0.793582,0.802388,0.767543


#### Experiment 2: Shuffle split

CNN

In [28]:
exdf = pd.read_csv("/home/csutter/DRIVE-clean/trackers_side_experiments/shuffle/nestcv_5cat_twotrain_OT0_m0_T1V2.csv")

exdf.groupby(["site","innerPhase"]).count()

Unnamed: 0.3  Unnamed: 0.2  Unnamed: 0.1  \
site               innerPhase                                              
NYSDOT_174002      NAOuterTest            82            82            82   
                   innerTest             156           156           156   
                   innerTrain            168           168           168   
                   innerVal               90            90            90   
NYSDOT_m4er5dez4ab NAOuterTest            76            76            76   
...                                      ...           ...           ...   
Skyline_6799       innerVal               92            92            92   
Skyline_8344       NAOuterTest            68            68            68   
                   innerTest             163           163           163   
                   innerTrain            167           167           167   
                   innerVal               76            76            76   

                                Unnamed: 0  img_name  img_orig  img_index  \
site               innerPhase                                               
NYSDOT_174002      NAOuterTest          82        82        82         82   
                   innerTest           156       156       156        156   
                   innerTrain          168       168       168        168   
                   innerVal             90        90        90         90   
NYSDOT_m4er5dez4ab NAOuterTest          76        76        76         76   
...                                    ...       ...       ...        ...   
Skyline_6799       innerVal             92        92        92         92   
Skyline_8344       NAOuterTest          68        68        68         68   
                   innerTest           163       163       163        163   
                   innerTrain          167       167       167        167   
                   innerVal             76        76        76         76   

                                img_cat   ID  Latitude  ...   yr   mo  day  \
site               innerPhase                           ...                  
NYSDOT_174002      NAOuterTest       82   82        82  ...   82   82   82   
                   innerTest        156  156       156  ...  156  156  156   
                   innerTrain       168  168       168  ...  168  168  168   
                   innerVal          90   90        90  ...   90   90   90   
NYSDOT_m4er5dez4ab NAOuterTest       76   76        76  ...   76   76   76   
...                                 ...  ...       ...  ...  ...  ...  ...   
Skyline_6799       innerVal          92   92        92  ...   92   92   92   
Skyline_8344       NAOuterTest       68   68        68  ...   68   68   68   
                   innerTest        163  163       163  ...  163  163  163   
                   innerTrain       167  167       167  ...  167  167  167   
                   innerVal          76   76        76  ...   76   76   76   

                                time  yr_mo  timeofday  timeofevent  \
site               innerPhase                                         
NYSDOT_174002      NAOuterTest    82     82         82           82   
                   innerTest     156    156        156          156   
                   innerTrain    168    168        168          168   
                   innerVal       90     90         90           90   
NYSDOT_m4er5dez4ab NAOuterTest    76     76         76           76   
...                              ...    ...        ...          ...   
Skyline_6799       innerVal       92     92         92           92   
Skyline_8344       NAOuterTest    68     68         68           68   
                   innerTest     163    163        163          163   
                   innerTrain    167    167        167          167   
                   innerVal       76     76         76           76   

                                innerPhaseOrig  foldnum  outerPhase  
site               innerPhase   

In [30]:
files_list = glob("/home/csutter/DRIVE-clean/side_experiments_data_and_models/CNN/data_preds_expShuffle/*")
files_list = [x for x in files_list if "A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue" in x]
# A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue
# A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue
print(len(files_list))

# df = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"] # change this according to phase
    dfs30.append(d)

df = pd.concat(dfs30).reset_index()

print(len(df))
display(df.head(4))

dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")

print(dict_i)

df = pd.DataFrame(dict_i,index=[0])


df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

display(df)


30
107445


,index,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,img_name,img_orig,site,img_index,...,innerPhase,outerPhase,prob_dry,prob_poor_viz,prob_snow,prob_snow_severe,prob_wet,model_pred,tracker,model_prob
0,3624,3624,3624,2,2,2,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,3,...,innerVal,inner5CV,0.084490,0.000908,0.086620,0.005610,0.822372,wet,nestcv_5cat_twotrain_OT2_m1_T3V4,0.822372
1,3625,3625,3625,3,3,3,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,4,...,innerVal,inner5CV,0.070394,0.000421,0.117544,0.003871,0.807771,wet,nestcv_5cat_twotrain_OT2_m1_T3V4,0.807771
2,3626,3626,3626,9,9,9,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,10,...,innerVal,inner5CV,0.315038,0.007869,0.465308,0.113067,0.098719,snow,nestcv_5cat_twotrain_OT2_m1_T3V4,0.465308
3,3627,3627,3627,23,23,23,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,30,...,innerVal,inner5CV,0.016293,0.000630,0.916882,0.040369,0.025827,snow,nestcv_5cat_twotrain_OT2_m1_T3V4,0.916882


{'totalims': 107445, 'correctims': 86813, 'nims_snow_severe': 7635, 'correct_snow_severe': 6929, 'nims_snow': 14945, 'correct_snow': 11888, 'nims_wet': 42290, 'correct_wet': 32774, 'nims_dry': 38000, 'correct_dry': 30925, 'nims_poor_viz': 4575, 'correct_poor_viz': 4297, 'ok': 102220, 'exp_desc': '', 'exp_details': '', 'tracker': ''}


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,107445,86813,7635,6929,14945,11888,42290,32774,38000,30925,...,,,,0.807976,0.813816,0.939235,0.79545,0.907531,0.774982,0.846203


Downstream

In [31]:
files_list = glob("/home/csutter/DRIVE-clean/side_experiments_data_and_models/downstream/data_preds_expShuffle/*")
files_list = [x for x in files_list if "A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue" in x]
# A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue
# A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse
# A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue
print(len(files_list))

# df = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"]# change this according to phase
    d["model_pred"] = d["ds_pred"]
    dfs30.append(d)

df = pd.concat(dfs30).reset_index()

print(len(df))
display(df.head(4))

dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")

print(dict_i)

df = pd.DataFrame(dict_i,index=[0])


df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

display(df)


30
107445


,index,Unnamed: 0.1,innerPhase,outerPhase,img_name,img_orig,site,img_cat,foldnum,timeofday,...,isobaricInhPa,uavg,ds_prob_dry,ds_prob_poor_viz,ds_prob_snow,ds_prob_snow_severe,ds_prob_wet,ds_pred,ds_prob,model_pred
0,17903,17903,innerVal,inner5CV,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,wet,0,night,...,500.0,2.697061,0.013133,0.009358,0.038123,0.005591,0.933795,wet,0.933795,wet
1,17904,17904,innerVal,inner5CV,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,snow,0,night,...,500.0,3.700014,0.007652,0.000966,0.980783,0.002741,0.007858,snow,0.980783,snow
2,17905,17905,innerVal,inner5CV,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,snow,0,night,...,500.0,2.821712,0.017800,0.000000,0.900575,0.003830,0.077796,snow,0.900575,snow
3,17906,17906,innerVal,inner5CV,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,snow,0,day,...,500.0,4.364937,0.012113,0.000000,0.690782,0.018857,0.278247,snow,0.690782,snow


{'totalims': 107445, 'correctims': 93264, 'nims_snow_severe': 7635, 'correct_snow_severe': 6937, 'nims_snow': 14945, 'correct_snow': 12550, 'nims_wet': 42290, 'correct_wet': 34964, 'nims_dry': 38000, 'correct_dry': 34576, 'nims_poor_viz': 4575, 'correct_poor_viz': 4237, 'ok': 103649, 'exp_desc': '', 'exp_details': '', 'tracker': ''}


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,107445,93264,7635,6937,14945,12550,42290,34964,38000,34576,...,,,,0.868016,0.909895,0.92612,0.839746,0.908579,0.826768,0.882221


Downstream - High vs low quality

In [43]:
files_list = glob("/home/csutter/DRIVE-clean/side_experiments_data_and_models/downstream/data_preds_expShuffle/*")

files_list = [x for x in files_list if "A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue" in x]

#### High Quality
dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"]# change this according to phase
    d = d[~d["site"].isin(Low_quality_all)] # Note the ~
    d["model_pred"] = d["ds_pred"]
    dfs30.append(d)
df = pd.concat(dfs30).reset_index()

print(len(df))
dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")
df = pd.DataFrame(dict_i,index=[0])
df["overall"] = df["correctims"]/df["totalims"]
for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]
df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

print("HIGH QUALITY")
display(df)

#### Low Quality
dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"]# change this according to phase
    d = d[d["site"].isin(Low_quality_all)]
    d["model_pred"] = d["ds_pred"]
    dfs30.append(d)
df = pd.concat(dfs30).reset_index()

print(len(df))
dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")
df = pd.DataFrame(dict_i,index=[0])
df["overall"] = df["correctims"]/df["totalims"]
for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]
df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

print("LOW QUALITY")
display(df)

61685
HIGH QUALITY


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,61685,55635,5595,5266,8530,7491,23440,20132,20160,18988,...,,,,0.901921,0.941865,0.94899,0.878195,0.941197,0.858874,0.913824


45760
LOW QUALITY


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,45760,37629,2040,1671,6415,5059,18850,14832,17840,15588,...,,,,0.822312,0.873767,0.778862,0.78862,0.819118,0.786844,0.809442


#### Experiment 3: Halved

CNN

In [32]:
files_list = glob("/home/csutter/DRIVE-clean/side_experiments_data_and_models/CNN/data_preds_expHalved/*")
files_list = [x for x in files_list if "A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue" in x]
# A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue
# A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue
# A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse
print(len(files_list))

# df = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"] # change this according to phase
    dfs30.append(d)

df = pd.concat(dfs30).reset_index()

print(len(df))
display(df.head(4))

dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")

print(dict_i)

df = pd.DataFrame(dict_i,index=[0])


df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

display(df)


30
56370


,index,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,innerPhase,Unnamed: 0,outerPhase,img_name,img_orig,...,timeofevent,innerPhaseOrig,prob_dry,prob_poor_viz,prob_snow,prob_snow_severe,prob_wet,model_pred,tracker,model_prob
0,0,0,0,0,0,innerVal,0,inner5CV,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,...,"dawn:12:06:29.361832, sunrise:12:37:47.866183,...",innerVal,0.087075,0.000871,0.005551,0.000645,0.905857,wet,nestcv_5cat_twotrain_OT2_m1_T3V4,0.905857
1,1,1,1,1,1,innerVal,1,inner5CV,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,...,"dawn:12:06:29.361832, sunrise:12:37:47.866183,...",innerVal,0.199449,0.001900,0.031666,0.000743,0.766242,wet,nestcv_5cat_twotrain_OT2_m1_T3V4,0.766242
2,2,2,2,2,2,innerVal,2,inner5CV,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,...,"dawn:12:06:29.361832, sunrise:12:37:47.866183,...",innerVal,0.199789,0.001212,0.028075,0.001219,0.769705,wet,nestcv_5cat_twotrain_OT2_m1_T3V4,0.769705
3,3,3,3,3,3,innerVal,3,inner5CV,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,...,"dawn:12:06:29.361832, sunrise:12:37:47.866183,...",innerVal,0.176675,0.000611,0.028771,0.000544,0.793399,wet,nestcv_5cat_twotrain_OT2_m1_T3V4,0.793399


{'totalims': 56370, 'correctims': 38927, 'nims_snow_severe': 3615, 'correct_snow_severe': 2929, 'nims_snow': 8020, 'correct_snow': 4888, 'nims_wet': 22575, 'correct_wet': 15494, 'nims_dry': 19525, 'correct_dry': 13439, 'nims_poor_viz': 2635, 'correct_poor_viz': 2177, 'ok': 51708, 'exp_desc': '', 'exp_details': '', 'tracker': ''}


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,56370,38927,3615,2929,8020,4888,22575,15494,19525,13439,...,,,,0.690562,0.688297,0.826186,0.609476,0.810235,0.686334,0.724106


Downstream

In [33]:
files_list = glob("/home/csutter/DRIVE-clean/side_experiments_data_and_models/downstream/data_preds_expHalved/*")
files_list = [x for x in files_list if "A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue" in x]
# A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue
# A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse
# A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue
print(len(files_list))

# df = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"]# change this according to phase
    d["model_pred"] = d["ds_pred"]
    dfs30.append(d)

df = pd.concat(dfs30).reset_index()

print(len(df))
display(df.head(4))

dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")

print(dict_i)

df = pd.DataFrame(dict_i,index=[0])


df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

display(df)


30
56370


,index,Unnamed: 0.1,innerPhase,outerPhase,img_name,img_orig,site,img_cat,foldnum,timeofday,...,isobaricInhPa,uavg,ds_prob_dry,ds_prob_poor_viz,ds_prob_snow,ds_prob_snow_severe,ds_prob_wet,ds_pred,ds_prob,model_pred
0,459,459,innerVal,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,snow_severe,0,night,...,500.0,5.797924,0.010498,0.084364,0.256943,0.635375,0.012820,snow_severe,0.635375,snow_severe
1,460,460,innerVal,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,snow_severe,0,night,...,500.0,5.468119,0.008053,0.022992,0.235846,0.726402,0.006707,snow_severe,0.726402,snow_severe
2,461,461,innerVal,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,snow_severe,0,day,...,500.0,4.567119,0.093357,0.012118,0.505226,0.229658,0.159642,snow,0.505226,snow
3,462,462,innerVal,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,snow_severe,0,day,...,500.0,4.894132,0.152175,0.014285,0.374567,0.191473,0.267500,snow,0.374567,snow


{'totalims': 56370, 'correctims': 43693, 'nims_snow_severe': 3615, 'correct_snow_severe': 2655, 'nims_snow': 8020, 'correct_snow': 5668, 'nims_wet': 22575, 'correct_wet': 16471, 'nims_dry': 19525, 'correct_dry': 17192, 'nims_poor_viz': 2635, 'correct_poor_viz': 1707, 'ok': 53217, 'exp_desc': '', 'exp_details': '', 'tracker': ''}


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,56370,43693,3615,2655,8020,5668,22575,16471,19525,17192,...,,,,0.775111,0.880512,0.647818,0.706733,0.73444,0.729612,0.739823


Downstream - High vs low quality
- See above for final main file where site lists are defined

In [42]:
files_list = glob("/home/csutter/DRIVE-clean/side_experiments_data_and_models/downstream/data_preds_expHalved/*")

files_list = [x for x in files_list if "A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue" in x]

#### High Quality
dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"]# change this according to phase
    d = d[~d["site"].isin(Low_quality_all)] # Note the ~
    d["model_pred"] = d["ds_pred"]
    dfs30.append(d)
df = pd.concat(dfs30).reset_index()

print(len(df))
dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")
df = pd.DataFrame(dict_i,index=[0])
df["overall"] = df["correctims"]/df["totalims"]
for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]
df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

print("HIGH QUALITY")
display(df)

#### Low Quality
dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"]# change this according to phase
    d = d[d["site"].isin(Low_quality_all)]
    d["model_pred"] = d["ds_pred"]
    dfs30.append(d)
df = pd.concat(dfs30).reset_index()

print(len(df))
dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")
df = pd.DataFrame(dict_i,index=[0])
df["overall"] = df["correctims"]/df["totalims"]
for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]
df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

print("LOW QUALITY")
display(df)

32295
HIGH QUALITY


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,32295,27389,2560,2172,4120,3061,12585,10565,10550,9979,...,,,,0.848088,0.945877,0.65,0.742961,0.848437,0.839491,0.805353


24075
LOW QUALITY


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,24075,16304,1055,483,3900,2607,9990,5906,8975,7213,...,,,,0.677217,0.803677,0.612903,0.668462,0.45782,0.591191,0.626811


#### Experiment 4: Shuffled & halved

CNN

In [35]:
files_list = glob("/home/csutter/DRIVE-clean/side_experiments_data_and_models/CNN/data_preds_expShuffleAndHalved/*")
files_list = [x for x in files_list if "A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue" in x]
# A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue
# A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse
print(len(files_list))

# df = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"] # change this according to phase
    dfs30.append(d)

df = pd.concat(dfs30).reset_index()

print(len(df))
display(df.head(4))

dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")

print(dict_i)

df = pd.DataFrame(dict_i,index=[0])


df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

display(df)


30
56370


,index,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,img_name,img_orig,site,...,innerPhase,outerPhase,prob_dry,prob_poor_viz,prob_snow,prob_snow_severe,prob_wet,model_pred,tracker,model_prob
0,1937,1937,1937,3624,2,2,2,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,...,innerVal,inner5CV,0.136969,0.000839,0.077057,0.001809,0.783327,wet,nestcv_5cat_twotrain_OT2_m1_T3V4,0.783327
1,1938,1938,1938,3625,3,3,3,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,...,innerVal,inner5CV,0.167643,0.000459,0.095357,0.000952,0.735589,wet,nestcv_5cat_twotrain_OT2_m1_T3V4,0.735589
2,1939,1939,1939,3626,9,9,9,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,...,innerVal,inner5CV,0.228970,0.003044,0.641643,0.032308,0.094035,snow,nestcv_5cat_twotrain_OT2_m1_T3V4,0.641643
3,1940,1940,1940,3627,23,23,23,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,...,innerVal,inner5CV,0.009348,0.000131,0.934841,0.006416,0.049265,snow,nestcv_5cat_twotrain_OT2_m1_T3V4,0.934841


{'totalims': 56370, 'correctims': 45948, 'nims_snow_severe': 3615, 'correct_snow_severe': 3281, 'nims_snow': 8020, 'correct_snow': 6478, 'nims_wet': 22575, 'correct_wet': 17208, 'nims_dry': 19525, 'correct_dry': 16422, 'nims_poor_viz': 2635, 'correct_poor_viz': 2559, 'ok': 54111, 'exp_desc': '', 'exp_details': '', 'tracker': ''}


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,56370,45948,3615,3281,8020,6478,22575,17208,19525,16422,...,,,,0.815114,0.841076,0.971157,0.807731,0.907607,0.762259,0.857966


Downstream

In [36]:
files_list = glob("/home/csutter/DRIVE-clean/side_experiments_data_and_models/downstream/data_preds_expShuffleAndHalved/*")
files_list = [x for x in files_list if "A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue" in x]
# A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue
# A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse
print(len(files_list))

# df = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"]# change this according to phase
    d["model_pred"] = d["ds_pred"]
    dfs30.append(d)

df = pd.concat(dfs30).reset_index()

print(len(df))
display(df.head(4))

dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")

print(dict_i)

df = pd.DataFrame(dict_i,index=[0])


df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

display(df)


30
56370


,index,Unnamed: 0.1,innerPhase,outerPhase,img_name,img_orig,site,img_cat,foldnum,timeofday,...,isobaricInhPa,uavg,ds_prob_dry,ds_prob_poor_viz,ds_prob_snow,ds_prob_snow_severe,ds_prob_wet,ds_pred,ds_prob,model_pred
0,9365,9365,innerVal,inner5CV,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,wet,0,night,...,500.0,2.697061,0.013084,0.010326,0.025350,0.000000,0.951240,wet,0.951240,wet
1,9366,9366,innerVal,inner5CV,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,snow,0,night,...,500.0,3.700014,0.006308,0.005097,0.967092,0.009429,0.012074,snow,0.967092,snow
2,9367,9367,innerVal,inner5CV,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,snow,0,night,...,500.0,2.821712,0.037282,0.004497,0.823664,0.007623,0.126933,snow,0.823664,snow
3,9368,9368,innerVal,inner5CV,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,snow,0,day,...,500.0,4.364937,0.011187,0.000000,0.600750,0.008034,0.380029,snow,0.600750,snow


{'totalims': 56370, 'correctims': 48869, 'nims_snow_severe': 3615, 'correct_snow_severe': 3264, 'nims_snow': 8020, 'correct_snow': 6809, 'nims_wet': 22575, 'correct_wet': 18693, 'nims_dry': 19525, 'correct_dry': 17579, 'nims_poor_viz': 2635, 'correct_poor_viz': 2524, 'ok': 54687, 'exp_desc': '', 'exp_details': '', 'tracker': ''}


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,56370,48869,3615,3264,8020,6809,22575,18693,19525,17579,...,,,,0.866933,0.900333,0.957875,0.849002,0.902905,0.82804,0.887631


Downstream - High vs low quality

In [44]:
files_list = glob("/home/csutter/DRIVE-clean/side_experiments_data_and_models/downstream/data_preds_expShuffleAndHalved/*")

files_list = [x for x in files_list if "A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue" in x]

#### High Quality
dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"]# change this according to phase
    d = d[~d["site"].isin(Low_quality_all)] # Note the ~
    d["model_pred"] = d["ds_pred"]
    dfs30.append(d)
df = pd.concat(dfs30).reset_index()

print(len(df))
dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")
df = pd.DataFrame(dict_i,index=[0])
df["overall"] = df["correctims"]/df["totalims"]
for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]
df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

print("HIGH QUALITY")
display(df)

#### Low Quality
dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"]# change this according to phase
    d = d[d["site"].isin(Low_quality_all)]
    d["model_pred"] = d["ds_pred"]
    dfs30.append(d)
df = pd.concat(dfs30).reset_index()

print(len(df))
dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")
df = pd.DataFrame(dict_i,index=[0])
df["overall"] = df["correctims"]/df["totalims"]
for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]
df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

print("LOW QUALITY")
display(df)

32295
HIGH QUALITY


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,32295,29516,2560,2430,4120,3689,12585,11004,10550,9990,...,,,,0.91395,0.946919,0.968952,0.895388,0.949219,0.874374,0.92697


24075
LOW QUALITY


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,24075,19353,1055,834,3900,3120,9990,7689,8975,7589,...,,,,0.803863,0.845571,0.780645,0.8,0.790521,0.76967,0.797281
